# Calibration - SPOTPY
## Upper Olivares catchment - january, 2023
#### Paul Sandoval Quilodrán - https://github.com/SQPaul/Olivares

#### import packages

In [ ]:
import subprocess
import os
import time
import numpy as np
import pandas as pd
import openpyxl
import datetime
import spotpy
from configparser import ConfigParser

In [ ]:
class sphy_model(object):

    def __init__(self): #self,startTime,endTime
        #self.startTime= startTime
        #self.endTime = endTime
        
        return

    def get_obs(self):
        
        obs_path = r"P:\Projects\Olivares\Streamflow\qobs_2011.csv"
        obs = pd.read_csv(obs_path)
        obs = obs.iloc[:,1]
        obs = obs.values
        
        self.observations = obs

        return 

    def run_sphy(self,DDFG=None,TCrit=None,SnowSC=None,DDFS=None,Swfactor=None,kx=None):
        
        #Read config_file
        config_path = r"P:\Projects\Olivares\SPHY\cfg\sw\sphy_config_sw_SA.cfg"
        config_file = ConfigParser()
        config_file.read(config_path)

        params_to_iterate = [DDFG,TCrit,SnowSC,DDFS,Swfactor,kx] #vector with param values        
        
        #Change the param values in the config file THE MAGIC!!!!
        for p in range(len(params_to_iterate)):
            if p == 0:
                config_file["GLACIER"]["DDFG"] = str(params_to_iterate[p])
            elif p == 1:
                config_file["SNOW"]["TCrit"] = str(params_to_iterate[p])
            elif p == 2:
                config_file["SNOW"]["SnowSC"] = str(params_to_iterate[p])
            elif p == 3:
                config_file["SNOW"]["DDFS"] = str(params_to_iterate[p])
            elif p == 4:
                config_file["CLIMATE"]["Swfactor"] = str(params_to_iterate[p])
            elif p == 5:
                config_file["ROUTING"]["kx"] = str(params_to_iterate[p])
                
        with open(config_path, 'w') as conf: #Save the config file with the params edited
            config_file.write(conf)
        
        #run sphy
        os.system("python -m SPHY.main_sw -i P:\Projects\Olivares\SPHY\cfg\sw\sphy_config_sw_SA.cfg")
        
        #Read the sim data
        sim_path = r"P:\Projects\Olivares\SPHY\output\QTOTSubBasinTSS.tss"
        sim_read = pd.read_csv(sim_path,index_col=False,skiprows=4)
        sim = []
        for v in range(len(sim_read)):
            val = sim_read.iloc[v,0].strip()
            val = " ".join(val.split())
            sim.append(val)
        sim = pd.DataFrame(sim)
        sim.columns = ["name"]
        sim = sim["name"].str.split(" ",expand=True)
        sim = pd.DataFrame(sim.iloc[:,2]).astype(float)
        sim = sim.iloc[:,0]
        sim = sim.values

        return sim

class spotpy_setup(object):  
    def __init__(self):
        self.sphymodel = sphy_model() #vic_model(datastart,dataend) # routine to run model

        # model parameters to calibrate #('parameter',min_value,max_value) we have to define these values based on the nc
        #lower bound, upper bound, step size, initial value
        self.params = [spotpy.parameter.Uniform('DDFG',5,11),
                       spotpy.parameter.Uniform('TCrit',-10,18),
                       spotpy.parameter.Uniform('SnowSC',0.1,0.6),
                       spotpy.parameter.Uniform('DDFS',1,7),
                       spotpy.parameter.Uniform('Swfactor',0.6,1.1),
                       spotpy.parameter.Uniform('kx',0.7,0.99)]

        return
    
    def parameters(self):
        
        return spotpy.parameter.generate(self.params)

    def simulation(self,vector): #OK
        simulations = self.sphymodel.run_sphy(DDFG=vector[0],TCrit=vector[1],SnowSC=vector[2],DDFS=vector[3],Swfactor=vector[4],kx=vector[5])
        
        #print("VECTOR ES: ",vector)
        #print("SIMULATION ES: ", simulations)
        #print("f")
        return simulations
    
    def evaluation(self,evaldates=False): #self,evaldates=False
        self.sphymodel.get_obs()
        return self.sphymodel.observations
        #print("g")
        
        #obs_csv = "/home/phi/Desktop/Projects/Droughts_northernPatagonia/SPOTPY/cuenca_piloto/q_sim.csv"
        #obs = pd.read_csv(obs_csv)
        #obs = obs.iloc[:,[1]]
        #obs = obs.values
        #observation_phi = obs
        
        #print("OBSERVACIONES ES: ", observation_phi)
        #return observation_phi
        
    def objectivefunction(self,simulation,evaluation):
        #print(len(evaluation))
        #print(len(simulation))
        objectivefunction = -(spotpy.objectivefunctions.kge(evaluation,simulation)-1) #KGE
        #objectivefunction = -(spotpy.objectivefunctions.nashsutcliffe(evaluation, simulation)-1) #NASH
        #print("KGE: ",spotpy.objectivefunctions.kge(evaluation,simulation))
        return objectivefunction
        
def findBestSim(dbPath):
    csv = pd.read_csv(dbPath+'.csv')

    results = np.array(csv)

    likes = 1-np.abs(np.array(csv.like1))

    idx = likes.argmin() #np.nanargmin 


    i = results[idx,1]
    d = results[idx,2]
    dmax = results[idx,3]
    w = results[idx,4]
    ex = results[idx,5]
    dep = results[idx,6]
    rm = results[idx,7]
    lai = results[idx,8]
    alb = results[idx,9]
    #print("ESTE ES EL RESULTS", dmax)
    
    params = [i,d,dmax,w,ex,dep,rm,lai,alb]
    
    print("ESTE ES EL MEJOR", idx)
    return params

def kge(aa):
    csv_2 = pd.read_csv(aa+'.csv')
    results_2 = np.array(csv_2)
    likes_2 = 1-(np.abs(np.array(csv_2.like1)))
    idx_2 = likes_2.argmin()
    val = results_2[idx_2,0]
    
    return val
    
def runStats(sim,obs):
    nse = 1 - (np.nansum((sim-obs)**2)/np.nansum((obs-obs.mean())**2))
    bias = np.nanmean(sim-obs)
    rmse = np.nanmean(np.sqrt((sim-obs)**2))
    kge = 1-np.sqrt((np.corrcoef(sim,obs)-1)**2+((sim.mean()/obs.mean())-1)**2+(((sim.var()/sim.mean())/(obs.var()/obs.mean()))-1)**2)
    #kge = -(spotpy.objectivefunction.kge(obs,sim)-1) #probar kge

    #print("j")
    return nse, bias, rmse, kge

def calibrate():
    # calibration setup object
    #cal_setup = spotpy_setup() !!!  
    #print("1.1 - cal_setup")
    # ../output/SCEUA_VIC_Nyando
    outCal = r"P:\Projects\Olivares\Calibration\SPOTPY\SCEUA_OLIVARES" #print("1.2 - outCal")
    # initialize calibration algorithm with
    sampler = spotpy.algorithms.sceua(spotpy_setup(),dbname=outCal,dbformat='csv') #    #print("1 - sampler")
    results = [] # mpty list to append iteration results !!!    #print("2.1 - sampler")
    # run calibration process
    sampler.sample(3) 
    #print ("Number of arguments:", len(sys.argv))
    #sampler.sample(int(sys.argv[1])),ngs=5)    
    #print("2.1 - sampler")
    
    results.append(sampler.getdata()) 
    #print("ESTE ES EL RESULTS2", results2)
    
    #params = findBestSim(outCal) #!!!
 
    #print('Best parameter set: {0}'.format(params)) #!!!
    
    #lastRun = vic_model() #vic_model(datetime.date(1990,1,1),datetime.date(1990,12,31)) !!!

    #sim = lastRun.run_vic(params[0],params[1],params[2],params[3],params[4],params[5],params[6],params[7],params[8]) #!!!

    #lastRun.get_obs() #!!!
    #obs = lastRun.observations #!!!

    #print(sim,obs) #!!!
    #stats = runStats(sim,obs) #!!!
    #print('Calibration error statistics...\n \
    #        NSE:{0}\tBias:{1}\tRMSE:{2}\KGE:{3}'.format(stats[0],stats[1],stats[2],stats[3])) #!!!
    
    #obj_funct = kge(outCal) #!!!
    
    
    #print("ESTE ES EL MEJOR", obj_funct) #!!!
    print("-------------------------------------- PSANDOVALQ -----------------------------------")
    print("------------------------  https://github.com/SQPaul/Olivares ------------------------")
    return

    
if __name__ == "__main__":
    calibrate()